<a href="https://colab.research.google.com/github/kunamneni117/kagglecomp/blob/master/Food_Reviews_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer,PorterStemmer

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [6]:
%ls '/gdrive/My Drive/Machine Learning/Amazon Reviews'

database.sqlite


In [0]:
connection = sqlite3.connect('/gdrive/My Drive/Machine Learning/Amazon Reviews/database.sqlite')
Amazon_Food_Reviews = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3""", connection) 

Amazon_Food_Reviews['Score'] = Amazon_Food_Reviews['Score'].apply(lambda x: 1 if x>3 else 0)

In [0]:
#Sorting data according to ProductId in ascending order
Amazon_Food_Reviews_Sorted_Data=Amazon_Food_Reviews.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [12]:
#Deduplication of entries
Amazon_Food_Reviews_Duplicates_Filtered=Amazon_Food_Reviews_Sorted_Data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
print("{} % of Data Retained in 100%".format((Amazon_Food_Reviews_Duplicates_Filtered.shape[0]/Amazon_Food_Reviews_Sorted_Data.shape[0])*100))

69.25890143662969 % of Data Retained in 100%


In [13]:
Amazon_Food_Reviews_Preprocessed=Amazon_Food_Reviews_Duplicates_Filtered[Amazon_Food_Reviews_Duplicates_Filtered.HelpfulnessNumerator<=Amazon_Food_Reviews_Duplicates_Filtered.HelpfulnessDenominator]
print("{} % of Data Retained in 100%".format((Amazon_Food_Reviews_Preprocessed.shape[0]/Amazon_Food_Reviews_Duplicates_Filtered.shape[0])*100))

99.99945081046646 % of Data Retained in 100%


In [0]:
from sklearn.model_selection import train_test_split
Amazon_Food_Reviews_Preprocessed_Filtered_columns=Amazon_Food_Reviews_Preprocessed[['Text','Score']].head(10000)
Train_Data,Test_Data= train_test_split(Amazon_Food_Reviews_Preprocessed_Filtered_columns,test_size=0.25,random_state=42)

In [33]:
print('{}% of Class 1 present in the 100% Data'.format((Train_Data[Train_Data.iloc[:,1]==1].shape[0]/Train_Data.shape[0])*100))

83.81333333333333% of Class 1 present in the 100% Data


In [18]:
def Decontracted(Phrase):
    Phrase=re.sub(r"won\'t","will not",Phrase)
    Phrase=re.sub(r"can\'t","can not",Phrase)
    Phrase=re.sub(r"\'re","are",Phrase)
    Phrase=re.sub(r"\'m","am",Phrase)
    Phrase=re.sub(r"\'t","not",Phrase)
    Phrase=re.sub(r"\'ll","will",Phrase)
    Phrase=re.sub(r"\'d","would",Phrase)
    Phrase=re.sub(r"\'s","is",Phrase)
    Phrase=re.sub(r"\'ve","have",Phrase)
    return Phrase
nltk.download('stopwords')
Stopwords = [word for word in stopwords.words('english') if word not in ['not','no']]

def Preprocessing_Texts(documents):
    Preprocessed=[]
    for sentence in tqdm(documents):
        sentence=re.sub(r"http\S+","",sentence)
        sentence= BeautifulSoup(sentence,'lxml').get_text()
        sentence=Decontracted(sentence)
        sentence= re.sub("\S*\d\S*","",sentence).strip()
        sentence= re.sub('[^a-zA-Z]+'," ",sentence)
        sentence=' '.join(word.lower() for word in sentence.split() if word.lower() not in Stopwords)
        Preprocessed.append(sentence)
    return Preprocessed

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [56]:
Train_Data_Preprocessed_Text=Preprocessing_Texts(Train_Data['Text'].values)
Test_Data_Preprocessed_Text=Preprocessing_Texts(Test_Data['Text'].values)

100%|██████████| 2500/2500 [00:01<00:00, 1920.45it/s]


In [20]:
#BOW
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() 
count_vect.fit(Train_Data_Preprocessed_Text)
print("some feature names ", count_vect.get_feature_names()[:10])
print('='*50)

Preprocessed_Text_Count_Vector = count_vect.transform(Train_Data_Preprocessed_Text)
print("the type of count vectorizer ",type(Preprocessed_Text_Count_Vector))
print("the shape of out text BOW vectorizer ",Preprocessed_Text_Count_Vector.get_shape())
print("the number of unique words ", Preprocessed_Text_Count_Vector.get_shape()[1])

some feature names  ['aa', 'aaaaa', 'aaaaaahhhhhyaaaaaa', 'aaaallll', 'aaah', 'aaahhhhhh', 'aahing', 'abandon', 'abandoned', 'abbey']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (7500, 20001)
the number of unique words  20001


In [0]:
Test_Preprocessed_Text_Count_Vector=count_vect.transform(Test_Data_Preprocessed_Text)

In [21]:
#BOW NGRAM
count_vect_ngram = CountVectorizer(ngram_range=(1,2),min_df=10) 
Preprocessed_Text_Count_Vector_ngram=count_vect_ngram.fit_transform(Train_Data_Preprocessed_Text)
print("the shape of out text BOW vectorizer NGRAM",Preprocessed_Text_Count_Vector_ngram.get_shape())
print("the number of unique words ", Preprocessed_Text_Count_Vector_ngram.get_shape()[1])

the shape of out text BOW vectorizer NGRAM (7500, 4972)
the number of unique words  4972


In [0]:
Test_Preprocessed_Text_Count_Vector_ngram=count_vect_ngram.transform(Test_Data_Preprocessed_Text)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf_idf_vect.fit(Train_Data_Preprocessed_Text)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

Preprocessed_Text_tf_idf_vect = tf_idf_vect.transform(Train_Data_Preprocessed_Text)
print("the type of count vectorizer ",type(Preprocessed_Text_tf_idf_vect))
print("the shape of out text TFIDF vectorizer ",Preprocessed_Text_tf_idf_vect.get_shape())
print("the number of unique words including both unigrams and bigrams ", Preprocessed_Text_tf_idf_vect.get_shape()[1])

some sample features(unique words in the corpus) ['ability', 'able', 'able buy', 'able find', 'able get', 'absolute', 'absolute favorite', 'absolutely', 'absolutely delicious', 'absolutely love']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (7500, 4972)
the number of unique words including both unigrams and bigrams  4972


In [0]:
Test_Preprocessed_Text_tf_idf_vect=tf_idf_vect.transform(Test_Data_Preprocessed_Text)

In [0]:
from gensim.models import Word2Vec
i=0
list_of_sentance=[]
for sentance in Train_Data_Preprocessed_Text:
    list_of_sentance.append(sentance.split())
word2vec_model= Word2Vec(list_of_sentance,min_count=5,size=50, workers=7)

In [24]:
w2v_words = list(word2vec_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  5896
sample words  ['purchased', 'product', 'shortly', 'diagnosed', 'type', 'ii', 'diabetes', 'agree', 'reviewers', 'concerning', 'taste', 'texture', 'syrup', 'itis', 'amazing', 'us', 'follow', 'diet', 'understanding', 'maltitol', 'used', 'gi', 'table', 'sugar', 'means', 'sharp', 'rise', 'blood', 'noted', 'consuming', 'even', 'normal', 'serving', 'size', 'first', 'time', 'upset', 'test', 'results', 'hours', 'eating', 'pancake', 'mix', 'second', 'result', 'no', 'french', 'toast', 'trying', 'use']


In [0]:
Avg_Word2Vec_Vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in list_of_sentance: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = word2vec_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    Avg_Word2Vec_Vectors.append(sent_vec)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
model = TfidfVectorizer()
tf_idf_matrix = model.fit_transform(Train_Data_Preprocessed_Text)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [0]:
# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

Tfidf_Word2Vec_Vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in list_of_sentance: # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = word2vec_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    Tfidf_Word2Vec_Vectors.append(sent_vec)
    row += 1

In [29]:
print(len(Tfidf_Word2Vec_Vectors))
print(len(Tfidf_Word2Vec_Vectors[0]))

7500
50


In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[1,2,4,5,7,10]}
knn=KNeighborsClassifier(n_jobs=-1,algorithm='brute')
grid_search_cv_count_vector=GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='precision')
grid_search_cv_count_vector.fit(Preprocessed_Text_Count_Vector.toarray(),Train_Data['Score'].values.reshape(-1,1))

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='brute', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 4, 5, 7, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='precision', verbose=0)

In [36]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[1,2,4,5,7,10]}
knn=KNeighborsClassifier(n_jobs=-1,algorithm='brute')
grid_search_cv_count_vector_ngram=GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='precision')
grid_search_cv_count_vector_ngram.fit(Preprocessed_Text_Count_Vector_ngram.toarray(),Train_Data['Score'].values.reshape(-1,1))

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='brute', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 4, 5, 7, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='precision', verbose=0)

In [37]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[1,2,4,5,7,10]}
knn=KNeighborsClassifier(n_jobs=-1,algorithm='brute')
grid_search_TFIDF=GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='precision')
grid_search_TFIDF.fit(Preprocessed_Text_tf_idf_vect.toarray(),Train_Data['Score'].values.reshape(-1,1))

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='brute', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 4, 5, 7, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='precision', verbose=0)

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[1,2,4,5,7,10]}
knn=KNeighborsClassifier(n_jobs=-1,algorithm='brute')
grid_search_AVG_Word2Vec=GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='precision')
grid_search_AVG_Word2Vec.fit(Avg_Word2Vec_Vectors,Train_Data['Score'].values.reshape(-1,1))

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='brute', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 4, 5, 7, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='precision', verbose=0)

In [39]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[1,2,4,5,7,10]}
knn=KNeighborsClassifier(n_jobs=-1,algorithm='brute')
grid_search_TFIDF_Word2Vec=GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='precision')
grid_search_TFIDF_Word2Vec.fit(Tfidf_Word2Vec_Vectors,Train_Data['Score'].values.reshape(-1,1))

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='brute', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 4, 5, 7, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='precision', verbose=0)

In [42]:
#print("Best Score In Count Vectorization",grid_search_cv_count_vector.best_score_)
print("Best Score In Count Vectorization NGRAM",grid_search_cv_count_vector_ngram.best_score_)
print("Best Score In TFIDF Vectorization",grid_search_TFIDF.best_score_)
print("Best Score In Avg Word2Vec Vectorization",grid_search_AVG_Word2Vec.best_score_)
print("Best Score In TFIDF Word2Vec Vectorization",grid_search_TFIDF_Word2Vec.best_score_)

Best Score In Count Vectorization NGRAM 0.8810120690649232
Best Score In TFIDF Vectorization 0.9241504275811419
Best Score In Avg Word2Vec Vectorization 0.8734231030646831
Best Score In TFIDF Word2Vec Vectorization 0.8644947279476946


In [45]:
print("Best Parameters In Count Vectorization NGRAM",grid_search_cv_count_vector_ngram.best_params_)
print("Best Parameters In TFIDF Vectorization",grid_search_TFIDF.best_params_)
print("Best Parameters In Avg Word2Vec Vectorization",grid_search_AVG_Word2Vec.best_params_)
print("Best Parameters In TFIDF Word2Vec Vectorization",grid_search_TFIDF_Word2Vec.best_params_)

Best Parameters In Count Vectorization NGRAM {'n_neighbors': 2}
Best Parameters In TFIDF Vectorization {'n_neighbors': 2}
Best Parameters In Avg Word2Vec Vectorization {'n_neighbors': 2}
Best Parameters In TFIDF Word2Vec Vectorization {'n_neighbors': 2}


In [47]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[1,2,4,5,7,10]}
knn=KNeighborsClassifier(n_jobs=-1,algorithm='brute')
grid_search_cv_count_vector=GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='f1')
grid_search_cv_count_vector.fit(Preprocessed_Text_Count_Vector.toarray(),Train_Data['Score'].values.reshape(-1,1))

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='brute', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 4, 5, 7, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [49]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[1,2,4,5,7,10]}
knn=KNeighborsClassifier(n_jobs=-1,algorithm='brute')
grid_search_cv_count_vector_ngram=GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='f1')
grid_search_cv_count_vector_ngram.fit(Preprocessed_Text_Count_Vector_ngram.toarray(),Train_Data['Score'].values.reshape(-1,1))

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='brute', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 4, 5, 7, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [50]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[1,2,4,5,7,10]}
knn=KNeighborsClassifier(n_jobs=-1,algorithm='brute')
grid_search_TFIDF=GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='f1')
grid_search_TFIDF.fit(Preprocessed_Text_tf_idf_vect.toarray(),Train_Data['Score'].values.reshape(-1,1))

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='brute', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 4, 5, 7, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [51]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[1,2,4,5,7,10]}
knn=KNeighborsClassifier(n_jobs=-1,algorithm='brute')
grid_search_AVG_Word2Vec=GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='f1')
grid_search_AVG_Word2Vec.fit(Avg_Word2Vec_Vectors,Train_Data['Score'].values.reshape(-1,1))

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='brute', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 4, 5, 7, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [52]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[1,2,4,5,7,10]}
knn=KNeighborsClassifier(n_jobs=-1,algorithm='brute')
grid_search_TFIDF_Word2Vec=GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='f1')
grid_search_TFIDF_Word2Vec.fit(Tfidf_Word2Vec_Vectors,Train_Data['Score'].values.reshape(-1,1))

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='brute', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 4, 5, 7, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [53]:
print("Best Score In Count Vectorization",grid_search_cv_count_vector.best_score_)
print("Best Score In Count Vectorization NGRAM",grid_search_cv_count_vector_ngram.best_score_)
print("Best Score In TFIDF Vectorization",grid_search_TFIDF.best_score_)
print("Best Score In Avg Word2Vec Vectorization",grid_search_AVG_Word2Vec.best_score_)
print("Best Score In TFIDF Word2Vec Vectorization",grid_search_TFIDF_Word2Vec.best_score_)

Best Score In Count Vectorization 0.912624218709225
Best Score In Count Vectorization NGRAM 0.9120746053098483
Best Score In TFIDF Vectorization 0.9120719686803427
Best Score In Avg Word2Vec Vectorization 0.9037627680549076
Best Score In TFIDF Word2Vec Vectorization 0.9032133134790855


In [55]:
print("Best Score In Count Vectorization",grid_search_cv_count_vector.best_params_)
print("Best Parameters In Count Vectorization NGRAM",grid_search_cv_count_vector_ngram.best_params_)
print("Best Parameters In TFIDF Vectorization",grid_search_TFIDF.best_params_)
print("Best Parameters In Avg Word2Vec Vectorization",grid_search_AVG_Word2Vec.best_params_)
print("Best Parameters In TFIDF Word2Vec Vectorization",grid_search_TFIDF_Word2Vec.best_params_)

Best Score In Count Vectorization {'n_neighbors': 10}
Best Parameters In Count Vectorization NGRAM {'n_neighbors': 7}
Best Parameters In TFIDF Vectorization {'n_neighbors': 7}
Best Parameters In Avg Word2Vec Vectorization {'n_neighbors': 10}
Best Parameters In TFIDF Word2Vec Vectorization {'n_neighbors': 10}


In [60]:
knn_cnt_vector=KNeighborsClassifier(n_jobs=-1,algorithm='brute',n_neighbors=10)
knn_cnt_vector.fit(Preprocessed_Text_Count_Vector,Train_Data['Score'])
Y_Pred=knn_cnt_vector.predict(Test_Preprocessed_Text_Count_Vector)
from sklearn.metrics import f1_score
print(f1_score(Test_Data['Score'],Y_Pred))

0.9165023011176857


In [64]:
knn_cnt_vector_ngram=KNeighborsClassifier(n_jobs=-1,algorithm='brute',n_neighbors=7)
knn_cnt_vector_ngram.fit(Preprocessed_Text_Count_Vector_ngram,Train_Data['Score'])
Y_Pred=knn_cnt_vector_ngram.predict(Test_Preprocessed_Text_Count_Vector_ngram)
print(f1_score(Test_Data['Score'],Y_Pred))

0.9139218269653052


In [65]:
knn_tfidf=KNeighborsClassifier(n_jobs=-1,algorithm='brute',n_neighbors=7)
knn_tfidf.fit(Preprocessed_Text_tf_idf_vect,Train_Data['Score'])
Y_Pred=knn_tfidf.predict(Test_Preprocessed_Text_tf_idf_vect)
print(f1_score(Test_Data['Score'],Y_Pred))

0.9153645833333334
